In [ ]:
import cv2
import numpy as np
import os
from sklearn.preprocessing import LabelBinarizer

In [ ]:
# Constants
IMG_SIZE = (100, 100)
DATA_DIR = '/content/drive/MyDrive/fruits-360/Training'

In [ ]:
# Function to preprocess images
def load_and_preprocess_data(data_dir):
    X, y = [], []
    classes = os.listdir(data_dir)
    label_binarizer = LabelBinarizer()
    labels = label_binarizer.fit_transform(classes)

    for idx, fruit in enumerate(classes):
        fruit_dir = os.path.join(data_dir, fruit)   ## for example, if data_dir is "data/fruits" and fruit is "apple", then fruit_dir would be "data/fruits/apple".
        for img in os.listdir(fruit_dir):
            img_path = os.path.join(fruit_dir, img)   ##For example, if fruit_dir is "data/fruits/apple" and img is "image1.jpg", then img_path would be "data/fruits/apple/image1.jpg".
            image = cv2.imread(img_path)
            image = cv2.resize(image, IMG_SIZE)
            image = image / 255.0  # Normalize
            X.append(image)
            y.append(labels[idx])

    return np.array(X), np.array(y), label_binarizer


In [ ]:
# Load and preprocess data
X, y, lb = load_and_preprocess_data(DATA_DIR)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:

# Define CNN model
def build_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(len(lb.classes_), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Build and train CNN model
input_shape = (100, 100, 3)
cnn_model = build_cnn_model(input_shape)
cnn_model.fit(X, y, epochs=20, validation_split=0.2)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
357/357 ━━━━━━━━━━━━━━━━━━━━ 14s 24ms/step - accuracy: 0.7492 - loss: 0.9688 - val_accuracy: 3.5014e-04 - val_loss: 36.2460
Epoch 2/20
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 1.0000 - loss: 2.7484e-04 - val_accuracy: 3.5014e-04 - val_loss: 40.0900
Epoch 3/20
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 1.0000 - loss: 1.0739e-04 - val_accuracy: 3.5014e-04 - val_loss: 42.3743
Epoch 4/20
357/357 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 1.0000 - loss: 6.7639e-05 - val_accuracy: 3.5014e-04 - val_loss: 44.0056
Epoch 5/20
357/357 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 1.0000 - loss: 3.8650e-05 - val_accuracy: 3.5014e-04 - val_loss: 45.3974
Epoch 6/20
357/357 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 1.0000 - loss: 2.8371e-05 - val_accuracy: 3.5014e-04 - val_loss: 46.6237
Epoch 7/20
357/357 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 1.0000 - loss: 2.0408e-05 - val_accuracy: 3.5014e-04 - val_loss: 47.6948
Epoch 8/20
357/357 ━━━━━━━━━━━━━━━━━━

In [ ]:
# Save the CNN model
cnn_model.save('cnn_fruit_model.h5')
import pickle
# Save the Label Binarizer
with open('label_binarizer.pkl', 'wb') as file:
    pickle.dump(lb, file)

In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load the enhanced Excel dataset
data = pd.read_csv('/content/fruits_updated.csv')

# Extract features and target
X_ann = data[['water (g)',	'protein (g)',	'total fat (g)',	'carbohydrates (g)',	'fiber (g)'	,'sugars (g)'	,'calcium (mg)',	'iron (mg)',	'magnessium (mg)',	'phosphorus (mg)',	'potassium (mg)',	'sodium (g)']].values
y_ann = data['energy (kcal/kJ)'].values.reshape(-1, 1)

# Standardize the features
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_ann_scaled = scaler_X.fit_transform(X_ann)
y_ann_scaled = scaler_y.fit_transform(y_ann)




In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the ANN model
def build_ann_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_ann_scaled.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

# Train the ANN model
ann_model = build_ann_model()
history = ann_model.fit(X_ann_scaled, y_ann_scaled, epochs=100, validation_split=0.2)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.4054 - mean_absolute_error: 0.5586 - val_loss: 3.1823 - val_mean_absolute_error: 1.1396
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step - loss: 0.3532 - mean_absolute_error: 0.5256 - val_loss: 2.9280 - val_mean_absolute_error: 1.0902
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.3059 - mean_absolute_error: 0.4926 - val_loss: 2.6865 - val_mean_absolute_error: 1.0401
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.2632 - mean_absolute_error: 0.4601 - val_loss: 2.4506 - val_mean_absolute_error: 0.9889
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.2246 - mean_absolute_error: 0.4276 - val_loss: 2.2291 - val_mean_absolute_error: 0.9388
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.1901 - mean_absolute_error: 0.3952 - val_loss: 2.0219 - val_mean_absolute_error: 0.8891
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.1599 - mean_absolute_error: 0.3632 - val_loss: 1.8291 - val_mean_

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Predict on test data
y_pred_scaled = ann_model.predict(X_ann_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_actual = scaler_y.inverse_transform(y_ann_scaled)

# Calculate metrics
mse = mean_squared_error(y_actual, y_pred)
mae = mean_absolute_error(y_actual, y_pred)
r2 = r2_score(y_actual, y_pred)


print(f"R² Score: {r2:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
R² Score: 0.93


In [ ]:
# Save the ANN model
ann_model.save('ann_calorie_model.h5')

In [ ]:
def predict_nutrition(fruit_name):
    # Get the index of the fruit
    fruit_data = data[data['name'] == fruit_name]
    if fruit_data.empty:
        return "Fruit not found in dataset"

    # Prepare ANN input features
    features = fruit_data[['water (g)',	'protein (g)',	'total fat (g)',	'carbohydrates (g)',	'fiber (g)'	,'sugars (g)'	,'calcium (mg)',	'iron (mg)',	'magnessium (mg)',	'phosphorus (mg)',	'potassium (mg)',	'sodium (g)']].values
    features_scaled = scaler_X.transform(features)

    # Predict calorie value using the ANN model
    calorie_prediction_scaled = ann_model.predict(features_scaled)
    calorie_prediction = scaler_y.inverse_transform(calorie_prediction_scaled)

    return {
        'Calorie (kcal)': round(calorie_prediction[0][0], 2),
        'Water (g)': fruit_data['water (g)'].values[0],
        'Protein (g)': fruit_data['protein (g)'].values[0],
        'Fiber (g)': fruit_data['fiber (g)'].values[0],
        'Sugar (g)': fruit_data['sugars (g)'].values[0],
        'Carbohydrates (g)': fruit_data['carbohydrates (g)'].values[0],
        'Total fat (g)': fruit_data['total fat (g)'].values[0]
    }


In [ ]:
import cv2
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

# Load pre-trained models
cnn_model = load_model('cnn_fruit_model.h5')
ann_model = load_model('ann_calorie_model.h5')

# Load Label Binarizer for decoding predictions
with open('label_binarizer.pkl', 'rb') as file:
    lb = pickle.load(file)

# Load Excel file with nutritional data
data = pd.read_csv('/content/fruits_updated.csv')

# Initialize scaler (same scaler used during ANN model training)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# Fit the scaler with nutritional data (excluding Fruit Name column)
features = data[['water (g)',	'protein (g)',	'total fat (g)',	'carbohydrates (g)',	'fiber (g)'	,'sugars (g)'	,'calcium (mg)',	'iron (mg)',	'magnessium (mg)',	'phosphorus (mg)',	'potassium (mg)',	'sodium (g)']]
calories = data[['energy (kcal/kJ)']]
scaler_X.fit(features)
scaler_y.fit(calories)

# Preprocess input image
def preprocess_image(image_path):
    IMG_SIZE = (100, 100)
    image = cv2.imread(image_path)
    image = cv2.resize(image, IMG_SIZE)
    image = image / 255.0
    return image.reshape(1, 100, 100, 3)

@tf.function
def predict_fruit(image):
  prediction = cnn_model.predict(image)  # Get the prediction
  return prediction[0]  # Return the first element (prediction for single image)



# Define a function for predicting diet recommendation


def predict_diet(image_path):
    # Step 1: Preprocess the image
    image = preprocess_image(image_path)

    # Step 2: Classify the fruit using CNN model
    fruit_pred = np.array(cnn_model.predict(image))  # Convert to NumPy array
    fruit_label_idx = np.argmax(fruit_pred)  # Find the index of the highest prediction score
    fruit_name = lb.classes_[fruit_label_idx]  # Get the fruit name from the label binarizer

    # Step 3: Retrieve nutritional data based on predicted fruit label
    if fruit_name not in data['name'].values:
        return f"Error: '{fruit_name}' not found in dataset. Please ensure folder names and Excel entries are consistent."

    fruit_data = data[data['name'] == fruit_name]
    features = fruit_data[['water (g)',	'protein (g)',	'total fat (g)',	'carbohydrates (g)',	'fiber (g)'	,'sugars (g)'	,'calcium (mg)',	'iron (mg)',	'magnessium (mg)',	'phosphorus (mg)',	'potassium (mg)',	'sodium (g)']].values
    features_scaled = scaler_X.transform(features)

    # Step 4: Refine calorie prediction using ANN model
    calorie_prediction_scaled = ann_model.predict(features_scaled)
    refined_calorie_value = scaler_y.inverse_transform(calorie_prediction_scaled)[0][0]

    # Step 5: Return predicted fruit and detailed nutritional information
    return {
        'Fruit Name': fruit_name,
        'Calorie (kcal)': round(refined_calorie_value, 2),
        'Water (g)': fruit_data['water (g)'].values[0],
        'Protein (g)': fruit_data['protein (g)'].values[0],
        'Fiber (g)': fruit_data['fiber (g)'].values[0],
        'Sugar (g)': fruit_data['sugars (g)'].values[0],
        'Carbohydrates (g)': fruit_data['carbohydrates (g)'].values[0],
        'Total fat (g)': fruit_data['total fat (g)'].values[0],
        'Sugars (g)': fruit_data['sugars (g)'].values[0],
        'Calcium (mg)': fruit_data['calcium (mg)'].values[0],
        'Iron (mg)': fruit_data['iron (mg)'].values[0],
        'Magnesium (mg)': fruit_data['magnessium (mg)'].values[0],
        'Phosphorus (mg)': fruit_data['phosphorus (mg)'].values[0],
        'Potassium (mg)': fruit_data['potassium (mg)'].values[0],
        'Sodium (g)': fruit_data['sodium (g)'].values[0]
    }


In [ ]:
# Example usage
result = predict_diet('watermelon1.jpg')
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
{'Fruit Name': 'Watermelon 1', 'Calorie (kcal)': 28.2, 'Water (g)': 91.45, 'Protein (g)': 0.61, 'Fiber (g)': 0.4, 'Sugar (g)': 6.2, 'Carbohydrates (g)': 7.55, 'Total fat (g)': 0.15, 'Sugars (g)': 6.2, 'Calcium (mg)': 7, 'Iron (mg)': 0.24, 'Magnesium (mg)': 10, 'Phosphorus (mg)': 11, 'Potassium (mg)': 112, 'Sodium (g)': 1}


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
result = predict_diet('pineapple.jpg')
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
{'Fruit Name': 'Pineapple 1', 'Calorie (kcal)': 53.05, 'Water (g)': 86.0, 'Protein (g)': 0.54, 'Fiber (g)': 1.4, 'Sugar (g)': 9.85, 'Carbohydrates (g)': 13.12, 'Total fat (g)': 0.12, 'Sugars (g)': 9.85, 'Calcium (mg)': 13, 'Iron (mg)': 0.29, 'Magnesium (mg)': 12, 'Phosphorus (mg)': 8, 'Potassium (mg)': 109, 'Sodium (g)': 1}


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
result = predict_diet('pear.jpg')
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
{'Fruit Name': 'Pear 1', 'Calorie (kcal)': 59.18, 'Water (g)': 83.71, 'Protein (g)': 0.38, 'Fiber (g)': 3.1, 'Sugar (g)': 9.8, 'Carbohydrates (g)': 15.46, 'Total fat (g)': 0.12, 'Sugars (g)': 9.8, 'Calcium (mg)': 9, 'Iron (mg)': 0.17, 'Magnesium (mg)': 7, 'Phosphorus (mg)': 11, 'Potassium (mg)': 119, 'Sodium (g)': 1}


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
